## 训练模型

### 引入第三方包

In [1]:
from PIL import Image
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.models import *
from keras.layers import *

import glob
import pickle
import os
import pydot_ng as pydot

import numpy as np
import tensorflow.gfile as gfile
import matplotlib.pyplot as plt


Using TensorFlow backend.


### 定义超参数和字符集

In [2]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
LOWERCASE = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
UPPERCASE = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
            'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

CAPTCHA_CHARSET = NUMBER  # 验证码字符集
CAPTCHA_LEN = 4           # 验证码长度
CAPTCHA_HEIGHT = 60       # 验证码高度
CAPTCHA_WIDTH = 160       # 验证码宽度

TRAIN_DATA_DIR = './train-data/'   # 验证码数据集目录
TEST_DATA_DIR = './test-data/'

BATCH_SIZE = 100   # 每一批图片处理的个数定义为 100 个
EPOCHS = 10        # 训练 10 轮就结束
OPT = 'adam'       # 带有自适应的功能，不需要我们去调 Learning Rate
LOSS = 'binary_crossentropy'

MODEL_DIR = './model/train_demo/'
MODEL_FORMAT = '.h5'
HISTORY_DIR = './history/train_demo'
HISTORY_FORMAT = '.history'

filename_str = '{}captcha_{}_{}_bs_{}_epochs_{}{}'

# 模型网络结构文件
MODEL_VIS_FILE = 'captcha_classification ' + '.png'
# 模型文件
MODEL_FILES = filename_str.format(MODEL_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS),
                                MODEL_FORMAT)
# 训练记录文件
HISTORY_FILE = filename_str.format(HISTORY_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS),
                                  HISTORY_FORMAT)

### 将 RGB 验证码转化为灰度图

In [3]:
def rgb2gray(img):
    # Y' = 0.299 R + 0.587 G + 0.114 B
    # https://en.wikipedia.ort/wiki/Grayscale#Converting_color_to_grayscale
    return np.dot(img[..., :3], [0.299, 0.587, 0.114])

### 对验证码中的每个字符进行 one-hot 编码

In [4]:
def text2vec(text, length = CAPTCHA_LEN, charset = CAPTCHA_CHARSET):
    text_len = len(text)
    # 验证码长度校验
    if text_len != length:
        raise ValueError('Error: length of captcha should be {}, but got {}'.format(length,
                                                                                   text_len))
    # 生成一个形如（CAPTCHA LEN*CAPTCHA_CHARTSET,）的一维向量
    # 例如，4个纯数字的验证码生成形如（4*10，）的一维向量
    vec = np.zeros(length * len(charset))
    for i in range(length):
        # One-hot 编码验证码中的每个数字
        # 每个字符的热码 = 索引 + 偏移量
        vec[charset.index(text[i]) + i*len(charset)] = 1
    return vec

### 将验证码向量解码为对应字符

In [5]:
def vec2text(vector):
    if not isinstance(vector, np.ndarray):
        vector = np.asarray(vector)
    vector = np.reshape(vector, [CAPTCHA_LEN, -1])
    text = ''
    for item in vector:
        text += CAPTCHA_CHARSET[np.argmax(item)]
    return text

### 适配 Keras 图像数据格式

In [6]:
def fit_keras_channels(batch, rows = CAPTCHA_HEIGHT, cols = CAPTCHA_WIDTH):
    if K.image_data_format() == 'channels_first':
        batch = batch.reshape(batch.shape[0], 1, rows, cols)
        input_shape = (1, rows, cols)
    else:
        batch = batch.reshape(batch.shape[0], rows, cols, 1)
        input_shape = (rows, cols, 1)
    return batch, input_shape

### 读取训练集

In [7]:
X_train = []
Y_train = []
for filename in glob.glob(TRAIN_DATA_DIR + '*.png'):
    X_train.append(np.array(Image.open(filename)))
    Y_train.append(filename.lstrip(TRAIN_DATA_DIR + '\\').rstrip('.png'))

### 处理训练集图像

In [8]:
# list -> rgb(numpy)
X_train = np.array(X_train, dtype = np.float32)
# rgb -> gray
X_train = rgb2gray(X_train)
# normalize
X_train = X_train / 255
# Fit keras channels
X_train, input_shape = fit_keras_channels(X_train)

print(X_train.shape, type(X_train))
print(input_shape)

(3948, 60, 160, 1) <class 'numpy.ndarray'>
(60, 160, 1)


### 处理训练集标签

In [9]:
Y_train = list(Y_train)

for i in range(len(Y_train)):
    Y_train[i] = text2vec(Y_train[i])
    
Y_train = np.asarray(Y_train)

print(Y_train.shape, type(Y_train))

(3948, 40) <class 'numpy.ndarray'>


### 读取测试集，处理对应图像和标签

In [10]:
X_test = []
Y_test = []
for filename in glob.glob(TEST_DATA_DIR + '*.png'):
    X_test.append(np.array(Image.open(filename)))
    Y_test.append(filename.lstrip(TEST_DATA_DIR + '\\').rstrip('.png'))
    
# list -> rgb -> gray -> normalization -> fit keras
X_test = np.array(X_test, dtype = np.float32)
X_test = rgb2gray(X_test)
X_test = X_test / 255
X_test, _ = fit_keras_channels(X_test)

Y_test = list(Y_test)
for i in range(len(Y_test)):
    Y_test[i] = text2vec(Y_test[i])
    
Y_test = np.array(Y_test)

print(X_test.shape, type(X_test))
print(Y_test.shape, type(Y_test))

(956, 60, 160, 1) <class 'numpy.ndarray'>
(956, 40) <class 'numpy.ndarray'>


### 创建验证码识别模型

In [11]:
# 输入层
inputs = Input(shape = input_shape, name = 'inputs')

# 第 1 层卷积
conv1 = Conv2D(32, (3, 3), name = 'conv1')(inputs)
relu1 = Activation('relu', name = 'relu1')(conv1)

# 第 2 层卷积
conv2 = Conv2D(32, (3, 3), name = 'conv2')(relu1)
relu2 = Activation('relu', name = 'relu2')(conv2)
pool2 = MaxPool2D(pool_size = (2, 2), padding = 'same', name = 'pool2')(relu2)

# 第 3 层卷积
conv3 = Conv2D(64, (3, 3), name = 'conv3')(pool2)
relu3 = Activation('relu', name = 'relu3')(conv3)
pool3 = MaxPool2D(pool_size = (2, 2), padding = 'same', name = 'pool3')(relu3)

# 将 Pooled feature map 摊平后输入全连接网络
x = Flatten()(pool3)

# Dropout
x =  Dropout(0.25)(x)

# 4 个全连接层分别做 10 分类， 分别对应 4 个字节
x = [Dense(10, activation = 'softmax', name = 'fc%d' % (i+1))(x) for i in range(4)]

# 4 个字符向量拼接在一起，与标签向量形式一致，作为模型输出
outs = Concatenate()(x)

# 定义模型的输入与输出
model = Model(inputs = inputs, output = outs)
model.compile(optimizer = OPT, loss = LOSS, metrics = ['accuracy'])

W1126 19:45:56.857612 11620 deprecation_wrapper.py:119] From e:\python\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1126 19:45:56.873655 11620 deprecation_wrapper.py:119] From e:\python\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1126 19:45:56.900727 11620 deprecation_wrapper.py:119] From e:\python\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1126 19:45:56.916769 11620 deprecation_wrapper.py:119] From e:\python\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1126 19:45:56.924790 11620 deprecation_wrapper.py:119] From e:\python\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name 

### 查看模型摘要

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 60, 160, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 58, 158, 32)  320         inputs[0][0]                     
__________________________________________________________________________________________________
relu1 (Activation)              (None, 58, 158, 32)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 56, 156, 32)  9248        relu1[0][0]                      
__________________________________________________________________________________________________
relu2 (Act

### 模型可视化

In [13]:
# to_file 这个参数显示的是绘制出的模型文件显示的位置
plot_model(model, to_file = MODEL_VIS_FILE, show_shapes = True)

### 训练模型

In [14]:
history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS,
                   verbose = 2, validation_data = (X_test, Y_test))

Train on 3948 samples, validate on 956 samples
Epoch 1/10
 - 50s - loss: 0.3301 - acc: 0.9000 - val_loss: 0.3255 - val_acc: 0.9000
Epoch 2/10
 - 47s - loss: 0.3247 - acc: 0.9000 - val_loss: 0.3240 - val_acc: 0.9000
Epoch 3/10
 - 49s - loss: 0.3165 - acc: 0.9000 - val_loss: 0.3061 - val_acc: 0.9002
Epoch 4/10
 - 45s - loss: 0.2735 - acc: 0.9045 - val_loss: 0.2621 - val_acc: 0.9087
Epoch 5/10
 - 47s - loss: 0.2235 - acc: 0.9187 - val_loss: 0.2355 - val_acc: 0.9160
Epoch 6/10
 - 50s - loss: 0.1883 - acc: 0.9312 - val_loss: 0.2206 - val_acc: 0.9211
Epoch 7/10
 - 47s - loss: 0.1501 - acc: 0.9450 - val_loss: 0.2003 - val_acc: 0.9274
Epoch 8/10
 - 47s - loss: 0.1141 - acc: 0.9588 - val_loss: 0.2027 - val_acc: 0.9299
Epoch 9/10
 - 47s - loss: 0.0856 - acc: 0.9696 - val_loss: 0.2051 - val_acc: 0.9309
Epoch 10/10
 - 49s - loss: 0.0628 - acc: 0.9784 - val_loss: 0.2181 - val_acc: 0.9297


### 预测样例

In [57]:
print(vec2text(Y_test[13]))

0107


In [58]:
yy = model.predict(X_test[13].reshape(1, 60, 160, 1))

In [59]:
print(vec2text(yy))

0107


### 保存模型

In [67]:
if not gfile.Exists(MODEL_DIR):
    gfile.MakeDirs(MODEL_DIR)

model.save(MODEL_FILES)
print('Saved trained model at %s ' % MODEL_FILES)

Saved trained model at ./model/train_demo/captcha_adam_binary_crossentropy_bs_100_epochs_10.h5 


### 保存训练过程记录

In [70]:
history.history['acc']

[0.8999999180028504,
 0.8999999180028504,
 0.9000062513737934,
 0.9045339050988658,
 0.9186866785255247,
 0.9311930043839877,
 0.9449531551914737,
 0.9588335940903081,
 0.9695541908192659,
 0.9784131154584064]

In [68]:
history.history.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])

In [69]:
if gfile.Exists(HISTORY_DIR) == False:
    gfile.MakeDirs(HISTORY_DIR)

with open(HISTORY_FILE, 'wb') as f:
    pickle.dump(history.history, f)

In [71]:
print(HISTORY_FILE)

./history/train_democaptcha_adam_binary_crossentropy_bs_100_epochs_10.history
